In [4]:
import pandas as pd
import pysal
import numpy as np
from esda.moran import Moran
from libpysal import weights

In [17]:
combined_df_no_nans = pd.read_parquet("/cluster/home/maikents/sinmod_features_masters/dataframes_pipeline_4/total_df_no_nans_and_labels.parquet")

In [6]:
print(combined_df_no_nans.columns)

Index(['x', 'y', 'bathymetry', 'slope', 'fine_BPI_std', 'broad_BPI_std',
       'log_ruggedness', 'aspect_cos', 'aspect_sin',
       'temperature_10th_percentile', 'temperature_mean',
       'temperature_90th_percentile', 'salinity_10th_percentile',
       'salinity_mean', 'salinity_90th_percentile', 'current_speed_mean',
       'current_speed_90th_percentile', 'statistical_northness',
       'statistical_eastness', 'current_aspect_angle', 'chlor_a_mean',
       'labels'],
      dtype='object')


In [5]:
#If no dim reduction: remove features after correlation analysis now

In [18]:
combined_df_no_nans = combined_df_no_nans.drop(columns=['temperature_10th_percentile','temperature_90th_percentile', 'salinity_10th_percentile','salinity_90th_percentile',
                                                       'current_speed_90th_percentile'], axis=1)

In [19]:
coral_df = combined_df_no_nans[['x', 'y', 'labels']]

In [20]:
absence_df = coral_df[coral_df['labels'] == 0]
downsampled_absences = absence_df.sample(n=718, random_state=42)

In [21]:
presence_df = coral_df[coral_df['labels'] == 1]

balanced_df = pd.concat([downsampled_absences, presence_df])

balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [22]:
labels = balanced_df['labels'].values.reshape(-1, 1)

In [23]:
coords = balanced_df[['x', 'y']].values

In [24]:
w = weights.KNN.from_array(coords, k=4)  # Create a KNN spatial weights matrix based on the coords
                                         # k=4 means that the matrix should include the 4 nearest
                                         # neighbors for each data point.

/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/libpysal/weights/distance.py:153: UserWarning: The weights matrix is not fully connected: 
 There are 3 disconnected components.
  W.__init__(self, neighbors, id_order=ids, **kwargs)


In [25]:
w.transform = 'r'

In [26]:
moran = Moran(labels, w)

print(f"Moran's I: {moran.I}")
print(f"p-value: {moran.p_sim}")

Moran's I: 0.6853618004558114
p-value: 0.001
